In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all

# Plots from external data
* Use the filter function to select data set you want to display

In [ ]:
def filter(mu, dmu, delta, g, dim):
    if dim != 3:
        return True
    #return False
    #if g != -2.88:
    #    return True
    #return False
    #if g !=-2.6:
    #    return True
    if delta != 2.5:
        return True
    if dmu != 3.11:
        return True
    return False

## Plot

In [ ]:
import os
import inspect
from os.path import join
import json
import glob
from json import dumps
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
pattern = join(currentdir,"data","FFState_[()d_0-9]*.json")
files = glob.glob(pattern)
plt.figure(figsize=(20,20))
style =['o','+','+']
gs = set()
for file in files:
    if os.path.exists(file):
        with open(file,'r') as rf:
            ret = json.load(rf)
            dim, mu, dmu, delta, g=ret['dim'], ret['mu'], ret['dmu'], ret['delta'], ret['g']
            gs.add(g)
            if filter(mu=mu, dmu=dmu, delta=delta, g=g, dim=dim):
                    continue
            print(file)
            datas = ret['data']
            dqs1, dqs2, ds1, ds2 = [],[],[],[]
            for data in datas:
                dq1, dq2, d = data
                if dq1 is not None:
                    dqs1.append(dq1)
                    ds1.append(d)
                if dq2 is not None:
                    dqs2.append(dq2)
                    ds2.append(d)
            plt.subplot(211)
            if len(ds1) < len(ds2):
                if len(ds1) > 0:
                    plt.plot(ds1, dqs1, style[dim-1], label=f"$\Delta=${delta}, $\mu$={mu}, $d\mu=${dmu}, g={g}")
            else:
                if len(ds2) > 0:
                    plt.plot(ds2, dqs2, style[dim-1], label=f"$\Delta=${delta}, $\mu$={mu}, $d\mu=${dmu}, g={g}")
            plt.subplot(212)
            if len(ds1) < len(ds2):
                if len(ds2) > 0:
                    plt.plot(ds2, dqs2, style[dim-1], label=f"$\Delta=${delta}, $\mu$={mu}, $d\mu=${dmu}, g={g}")
            else:
                if len(ds1)> 0:
                    plt.plot(ds1, dqs1, style[dim-1], label=f"$\Delta=${delta}, $\mu$={mu}, $d\mu=${dmu}, g={g}")
            #break
print(gs)    
plt.subplot(211)
plt.xlabel(f"$\Delta$")
plt.ylabel(f"$\delta q$")
plt.title(f"Lower Branch")
plt.legend()
plt.subplot(212)
plt.xlabel(f"$\Delta$")
plt.ylabel(f"$\delta q$")
plt.title(f"Upper Branch")
plt.legend()

## Compute Current and Pressure
$$
J=\int dk\left[(k+\delta q) f_a + (k - \delta q) f_b\right]
$$

In [ ]:
import os
import inspect
from os.path import join
import json
import glob
import operator
import warnings
warnings.filterwarnings("ignore")
from json import dumps
def PlotCurrentPressure(alignLowerBranches=True, alignUpperBranches=True, showLegend=False):
    currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
    pattern = join(currentdir,"..","mmf_hfb","data","FFState_J_P[()d_0-9]*")
    files=glob.glob(pattern)
    plt.figure(figsize=(20,20))
    gs = set()
    for file in files[0:]:
        if os.path.exists(file):
            with open(file,'r') as rf:
                ret = json.load(rf)
                dim, mu, dmu, delta, g=ret['dim'], ret['mu'], ret['dmu'], ret['delta'],ret['g']
                if filter(mu=mu, dmu=dmu, delta=delta, g=g, dim=dim):
                    continue                
                k_c = None
                if 'k_c' in ret:
                    k_c = ret['k_c']
                    print(f"k_c={k_c}")
                gs.add(g)
                print(file)    
               
                data1, data2 = ret['data']
                dqs1, dqs2, ds1, ds2, j1, j2, ja1, ja2, jb1, jb2, P1, P2 = [],[],[],[],[],[],[],[],[],[],[],[]
                for data in data1:
                    d, q, p, j, j_a, j_b = data['d'],data['q'],data['p'],data['j'],data['ja'],data['jb']
                    ds1.append(d)
                    dqs1.append(q)
                    j1.append(j)
                    ja1.append(j_a)
                    jb1.append(j_b)
                    P1.append(p)
                for data in data2:
                    d, q, p, j, j_a, j_b = data['d'],data['q'],data['p'],data['j'], data['ja'], data['jb']
                    ds2.append(d)
                    dqs2.append(q)
                    j2.append(j)
                    ja2.append(j_a)
                    jb2.append(j_b)
                    P2.append(p)

                plt.subplot(321)
                plt.plot(ds1, dqs1,"+", label=f"$\Delta=${delta},$\mu=${mu},$d\mu=${dmu}")
                plt.subplot(322)
                plt.plot(ds2, dqs2,"+", label=f"$\Delta=${delta},$\mu=${mu},$d\mu=${dmu}")
                plt.subplot(323)
                if len(P1) > 0:
                    if alignLowerBranches:
                        P1 = np.array(P1)
                        P1_ = P1- P1.min()
                    else:
                        P1_=P1
                    index1, value = max(enumerate(P1), key=operator.itemgetter(1))
                    plt.plot(ds1, P1_, label=f"$\Delta=${delta},$\mu=${mu},$d\mu=${dmu}")
                    plt.axvline(ds1[index1])

                plt.subplot(324)
                if len(P2) > 0:
                    if alignUpperBranches:
                        P2 = np.array(P2)
                        P2_ = P2 - P2.min()
                    else:
                        P2_ = P2
                    index2, value = max(enumerate(P2), key=operator.itemgetter(1))
                    plt.plot(ds2, P2_, label=f"$\Delta=${delta},$\mu=${mu},$d\mu=${dmu}")
                    plt.axvline(ds2[index2])
                    print(data2[index2])
                plt.subplot(325)
                plt.plot(ds1, j1, label=f"$j_p, \Delta=${delta},$\mu=${mu},$d\mu=${dmu}")
                plt.plot(ds1, ja1, label=f"j_a")
                plt.plot(ds1, jb1, label=f"j_b")
                plt.axvline(ds1[index1])
                plt.subplot(326)
                #plt.plot(ds2, j2, label=f"$j_p, \Delta=${delta},$\mu=${mu},$d\mu=${dmu}")
                plt.plot(ds2, ja2, "+",label=f"j_a")
                plt.plot(ds2, jb2, "+",label=f"j_b")
                plt.axvline(ds2[index2])
                plt.axhline(0)
                #break
        
    for i in range(1,7):
        plt.subplot(3,2,i)
        if showLegend:
                plt.legend()
        if i == 1:
            plt.title(f"Lower Branch")
            plt.ylabel("$\delta q$")
        if i == 2:
            plt.title(f"Upper Branch")
            plt.ylabel("$\delta q$")
        if i == 3 or i == 4:
            plt.ylabel("$Pressure$")
        if i == 5 or i == 6:
            plt.ylabel("$Current$")
        plt.xlabel("$\Delta$")

In [ ]:
PlotCurrentPressure(alignLowerBranches=True, alignUpperBranches=True, showLegend=True)